# Graphs

## Lib - Imports

In [1]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import networkx as nx
import re
from itertools import chain
import numpy as np
import holoviews as hv

## Dataset - Stellantis North America Value Optimization Ideas

In [5]:
MCM_df = pd.read_excel('/home/joaocosentino/mestrado_prof/data_viz/trees/eVOs_Global.xlsx')

In [6]:
columns_to_keep = ['Index',
       'Manager Group Code', 'Supplier Group Name','PSA / FCA Region',
       'Savings / Econs / Correction', 'Id_Idea', 'Metafunction', 'Function',
       'Technical Teams','Technical VO Group','Saving Type',
       'Short Description', 'CY Saving [€]',
       'avancement', 'Application Year', 'Application Month']
MCM_df.drop(columns=[col for col in MCM_df.columns if col not in columns_to_keep],inplace=True)

In [7]:
mcm_2023 = MCM_df[MCM_df["Application Year"]== 2023] # CY 2023
mcm_2023 = mcm_2023[mcm_2023['Savings / Econs / Correction'] == 'Savings'] #Savings only
mcm_2023 = mcm_2023[mcm_2023['avancement'] == "Implemented"]
mcm_2023 = mcm_2023[mcm_2023['Metafunction'].isin(['BODY, CHASSIS and INTERIOR','SOFTWARE X','PROPULSION SYSTEMS'])]
mcm_2023 = mcm_2023[mcm_2023['PSA / FCA Region'] == 'North America']
mcm_2023['CY Saving [€]'] = mcm_2023['CY Saving [€]'].astype(str).str.replace("-",'')
mcm_2023['CY Saving [€]'] = mcm_2023['CY Saving [€]'].astype(int)

### Criando Regex para capturar quais programas são afetados pelas ideias

In [8]:
acronyms_list = ["DT", "JL", "JT", "MP", "DJ", "DD", "DP", "VF","HD","HDCC","WL","WD",'RU',"WS",'LB']
pattern = r'\b(?:' + '|'.join(acronyms_list) + r')\b'
acronyms = list(map(lambda x: re.findall(pattern,x) if re.findall(pattern,x) else np.nan,mcm_2023['Short Description'].iloc[:]))


In [9]:
mcm_2023['Vehicle'] = acronyms
mcm_2023.dropna(inplace=True)

In [24]:
mcm_2023.head()

,Index,Manager Group Code,Supplier Group Name,PSA / FCA Region,Savings / Econs / Correction,Id_Idea,Metafunction,Function,Technical Teams,Technical VO Group,Saving Type,Short Description,CY Saving [€],avancement,Application Year,Application Month,Vehicle
21191,MCM_72355,ePWT & Powertrain Systems,DELTA ELECTRONICS INC,North America,Savings,MCM_72355,PROPULSION SYSTEMS,PROPULSION SYSTEMS,ELECTRIC DRIVE SYSTEMS,ePT - PHEV,Ecotech,Remove Waymo content from RU-PHEV IDCM (creati...,0,Implemented,2023,5,[RU]
21192,MCM_72355,ePWT & Powertrain Systems,DELTA ELECTRONICS INC,North America,Savings,MCM_72355,PROPULSION SYSTEMS,PROPULSION SYSTEMS,ELECTRIC DRIVE SYSTEMS,ePT - PHEV,Ecotech,Remove Waymo content from RU-PHEV IDCM (creati...,0,Implemented,2023,5,[RU]
21197,MCM_72355,ePWT & Powertrain Systems,DELTA ELECTRONICS INC,North America,Savings,MCM_72355,PROPULSION SYSTEMS,PROPULSION SYSTEMS,ELECTRIC DRIVE SYSTEMS,ePT - PHEV,Ecotech,Remove Waymo content from RU-PHEV IDCM (creati...,0,Implemented,2023,5,[RU]
21255,MCM_73611,Drivelines & Transmissions,"American Axle & Manufacturing Holdings, Inc.",North America,Savings,MCM_73611,PROPULSION SYSTEMS,PROPULSION SYSTEMS,TRANSMISSION & DRIVE LINE,Driveline & Axle,Ecotech,KL & RU RESOURCE THE PTU/RDM NEEDLE BEARING FR...,0,Implemented,2023,2,[RU]
21289,MCM_73611,Drivelines & Transmissions,"American Axle & Manufacturing Holdings, Inc.",North America,Savings,MCM_73611,PROPULSION SYSTEMS,PROPULSION SYSTEMS,TRANSMISSION & DRIVE LINE,Driveline & Axle,Ecotech,KL & RU RESOURCE THE PTU/RDM NEEDLE BEARING FR...,0,Implemented,2023,2,[RU]


In [79]:
functions = list(mcm_2023['Function'].unique())
veh = list(mcm_2023['Vehicle'].unique())
tech_teams = list(mcm_2023['Technical Teams'].unique())
tech_VO = list(mcm_2023['Technical VO Group'].unique())

In [62]:
mcm_2023['Vehicle'] = mcm_2023['Vehicle'].astype(str)

In [95]:
values = [mcm_2023[mcm_2023['Vehicle'] == vehicle].count().iloc[0] for vehicle in veh]
func = [mcm_2023[mcm_2023['Function'] == function].count().iloc[0] for function in functions]
tech = [mcm_2023[mcm_2023['Technical Teams'] == teams].count().iloc[0] for teams in tech_teams]

In [104]:
functions

['PROPULSION SYSTEMS',
 'CHASSIS',
 'BODY',
 'INTERIORS',
 'SOFTWARE X',
 'MECHATRONICS']

In [107]:
# Define the nodes
nodes = dict(label=list(mcm_2023['Function'].unique()),
             color=["blue", "green", "green", "green", "red", "yellow"])

# Define the links
links = dict(source=[0, 1, 2, 3, 4, 5],  # indices correspond to labels, eg. "A" is index 0
             target=[5, 2, 4, 2, 5],
             value=func)


fig = go.Figure(data=[go.Sankey(node=dict(label=nodes['label'], color=nodes['color']),
                                 link=dict(source=links['source'], target=links['target'], value=links['value']))])

fig.update_layout(title_text="Relação de Ideas entre Metafuncions", font_size=10)
fig.show()